In [ ]:
#install suds 
#Suds is a lightweight SOAP python client that provides a service proxy for Web Services.
!pip install suds
#install dependencies for creating pdf
!pip install domain-availability
!pip install pdfkit
!wget https://github.com/wkhtmltopdf/packaging/releases/download/0.12.6-1/wkhtmltox_0.12.6-1.bionic_amd64.deb
!cp wkhtmltox_0.12.6-1.bionic_amd64.deb /usr/bin
!sudo apt install /usr/bin/wkhtmltox_0.12.6-1.bionic_amd64.deb
!pip install PyPDF2
!pip install abydos
!pip install phonetics

In [ ]:
from suds.client import Client
import pdfkit
from IPython.core.display import HTML
import re
#try cases
from abydos.phonetic import FONEM, Phonet, Dolby, RussellIndex, Phonem, PHONIC
#russell_index_alpha
#from domainavailability import *
import time
from abydos.distance import sim
import pandas as pd
from PyPDF2 import PdfMerger
#function to search for company name and return dataframe
import json
import requests
from pandas import json_normalize
import unicodedata
import phonetics

In [ ]:
#connect to widl using required authentification
#to do create python environment to hide password and username
url = 'https://www.swissreg.ch/services11/SwissregWebService?wsdl'
client = Client(url, username='legalmarque', password='dvMXQsUzcIXEZMdAD6Ls')

In [ ]:
query = input("Quelle marque souhaitez-vous chercher ? ")

In [ ]:
#to do match queries without vowels
vowels = r'[AEIOUÀÄÈÉÊÖÜ]'

In [ ]:
#remove accents function
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [ ]:
def anti_vowel(s):
    """Remove vowels from string."""
    result = re.sub(vowels, '', s, flags=re.IGNORECASE)
    return result

In [ ]:
#string rotation
def vowel_rotate(string):
    """rotate vowels in query only if vowels exist"""
    results = []

    sub_function = lambda x: 'eiouaEIOUA'['aeiouAEIOU'.find(x.group(0))]

    for x in range(5):
        string = re.sub('(?i)[aeiou]', sub_function, string)
        results.append(string)
    return ",".join(results)


In [ ]:
pe_fr = FONEM()
pe_de = Phonet()
db = Dolby()
pe_gm = Phonem()
pe_ph = PHONIC()

In [ ]:
#search by query name according to documentation
Ip_Right = client.service.searchIpRight("CH-TM", 'tmText' '=' f'{vowel_rotate(query)} {query} {anti_vowel(query)} {db.encode_alpha(query)} {pe_gm.encode(query)} {pe_fr.encode(query)} ')

In [ ]:
#save query result in csv format
with open('ip_right.csv', 'w') as f:
    f.write(Ip_Right)

In [ ]:
#ascertain if result is same as IPI register
# Ipi_Ip_Right = list(Ip_Right.split(','))
# len(Ipi_Ip_Right)

In [ ]:
#use query result as input to generate both trademark & bname xml
response = client.service.getIpRightXML("CH-TM", Ip_Right)

In [ ]:
#save query response as xml
with open('response.xml', 'w') as f:
    f.write(response)

**Marque Visuelle**

In [ ]:
def retrieve_marque_data(transac):
    """"""
    
    basipikey = transac.find("basipikey").text
    origlan = transac.find("origlan").text
    basappn = transac.find("basappn").text
    basregn = transac.find("basregn").text if transac.find("basregn") is not None else None

    basannd = transac.find("basannd").text
    basstate = transac.find("basstate").text

    basregper = transac.find("basregper").text
    expirationdate = transac.find("expirationdate").text

    marpicn = transac.find("marpicn").text
    
#     print(f"Marque verbale: {markve}")
    
    # Classes
    gsgr = transac.find("gsgr")
    intreggs = gsgr.findall("intregg")

    classes = []

    for intregg in intreggs:
        
        nicclai = intregg.find("nicclai").text
        
        classes.append(int(nicclai))

    # Class description
    gstermt = gsgr.find("gstermt").text

    regadrs = transac.findall("regadr")
    
    # TODO check format of mandataire/titulaire
    titulaire_name = None
    titulaire_addrl = None
    titulaire_plainco = None
    titulaire_nat = None

    mandataire_namel = None
    mandataire_addrl = None
    mandataire_plainco = None
    mandataire_nat = None
    
    # Addresses
    for regadr in regadrs:
        
        addrrole = regadr.find("addrrole").text
        nameadd = regadr.find("nameadd")
        
        namel = nameadd.find("namel").text if nameadd.find("namel") is not None else None
        addrl = nameadd.find("addrl").text if nameadd.find("addrl") is not None else None
        
        plainco = nameadd.find("plainco").text
        nat = nameadd.find("nat").text
        
        # 1: titulaire
        if int(addrrole) == 1:
            titulaire_name = namel
            titulaire_addrl = addrl
            titulaire_plainco = plainco
            titulaire_nat = nat
        
        # 3: mandataire
        elif int(addrrole) == 3:
            mandataire_namel = namel
            mandataire_addrl = addrl
            mandataire_plainco = plainco
            mandataire_nat = nat
        
    # TODO verify what oppositiongr and oppositionstate mean
    oppositiongr = transac.find("oppositiongr")
    oppositionstate = oppositiongr.find("oppositionstate")

    martype = transac.find("martype").text
    
    # TODO: oppositiongr
    row = [basipikey, origlan, basappn,basregn, basannd, basstate,marpicn, basregper, expirationdate,
           classes, gstermt, 
           titulaire_name,
           titulaire_addrl,
           titulaire_plainco,
           titulaire_nat,
           mandataire_namel, 
           mandataire_addrl,
           mandataire_plainco,
           mandataire_nat,
           martype]
    
    return row

In [ ]:
def parse_xml(xml_file):
    import xml.etree.ElementTree as et 
    
    rows = []
    
    # Parse file
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()

    for node in xroot:
        if node.tag == 'transac':
            for transac in node:
                if transac.tag == 'marinfo':
                    # Marques verbales
                    marpicn = transac.find("marpicn")
                    if marpicn is not None:
                        row = retrieve_marque_data(transac)
                        rows.append(row)
    return rows

In [ ]:
tables = []
rows = parse_xml("/content/response.xml")
tables.extend(rows)

In [ ]:
#Display classes in ET module
columns = ['basipikey', 'origlan', 'basappn','basregn', 'basannd', 'basstate','marpicn', 'basregper', 'expirationdate',
           'classes', 'gstermt', 
           'titulaire_name',
           'titulaire_addrl',
           'titulaire_plainco',
           'titulaire_nat',
           'mandataire_namel', 
           'mandataire_addrl',
           'mandataire_plainco',
           'mandataire_nat',
           'martype']

In [ ]:
df = pd.DataFrame(tables, columns=columns)

In [ ]:
df.head()

##Visual Data Cleaning and Preparation

**Merge, Rename and drop unused Columns**

In [ ]:
df['titulaire'] = df.titulaire_name.fillna('') + ' ' + df.titulaire_addrl.fillna('') + ' ' + df.titulaire_plainco.fillna('') + ' ' + df.titulaire_nat.fillna('')

In [ ]:
# df['titulaire'] = df[['titulaire_name', 'titulaire_addrl', 'titulaire_plainco', 'titulaire_nat']].apply(lambda x: ' '.join(x), axis=1)

In [ ]:
df.drop(['basipikey', 'origlan',  'basannd', 'basstate', 'basregper', 'expirationdate', 'gstermt',
       'titulaire_name', 'titulaire_addrl', 'titulaire_plainco',
       'titulaire_nat', 'mandataire_namel', 'mandataire_addrl',
       'mandataire_plainco', 'mandataire_nat', 'martype' ], axis=1, inplace=True)

In [ ]:
#old,new
df.rename(columns = {'classes':'Classes de Nice', 'basappn':'No de la demande', 'basregn':'No de la marque','basannd':'Depot', 'marpicn':'Logo' }, inplace = True)

In [ ]:
# df['mandataire'] = df.mandataire_namel.fillna('') + ' ' + df.mandataire_addrl.fillna('') + ' ' + df.mandataire_plainco.fillna('') + ' ' + df.mandataire_nat.fillna('')

In [ ]:
df['titulaire'] = df['titulaire'].apply(lambda x: remove_accents(x))

In [ ]:
df['titulaire'] =  df['titulaire'].str.replace(r'\n', '', regex=True)
# df['mandataire'] =  df['mandataire'].str.replace(r'\n', '', regex=True)

In [ ]:
df['Classes de Nice'] = df['Classes de Nice'].apply(lambda x: ','.join(map(str, x)))

In [ ]:
#display trademark logo in dataframe and convert to html
def to_img_tag(path):
    return '<img src="'+ path + '" width="80"  >'


df_html = df.to_html(escape=False,formatters=dict(Logo=to_img_tag)).replace('<table border="1" class="dataframe">',
                                                              '<table border="1" style="border-collapse: '
                                                              'collapse;text-align: center;font-family:Calibri;font-size:10px;" '
                                                              'class="dataframe">').replace(
                                                              '<tr style="text-align: right;">', 
                                                              '<tr style="text-align: center;table-layout: fixed; width: auto; index:False";>')

In [ ]:
#save html result
with open("df.html", "w") as f:
    f.write(df_html)

In [ ]:
pdfkit.from_file('df.html', 'marque_logo.pdf')



**Marque Verbal**

In [ ]:
def retrieve_verbal_data(transac):
    """"""
    
    basipikey = transac.find("basipikey").text
    origlan = transac.find("origlan").text
    basappn = transac.find("basappn").text
    basregn = transac.find("basregn").text if transac.find("basregn") is not None else None

    basannd = transac.find("basannd").text
    basstate = transac.find("basstate").text

    basregper = transac.find("basregper").text
    expirationdate = transac.find("expirationdate").text

    markve = transac.find("markve").text
    
#     print(f"Marque verbale: {markve}")
    
    # Classes
    gsgr = transac.find("gsgr")
    intreggs = gsgr.findall("intregg")

    classes = []

    for intregg in intreggs:
        
        nicclai = intregg.find("nicclai").text
        
        classes.append(int(nicclai))

    # Class description
    gstermt = gsgr.find("gstermt").text

    regadrs = transac.findall("regadr")
    
    # TODO check format of mandataire/titulaire
    titulaire_name = None
    titulaire_addrl = None
    titulaire_plainco = None
    titulaire_nat = None

    mandataire_namel = None
    mandataire_addrl = None
    mandataire_plainco = None
    mandataire_nat = None
    
    # Addresses
    for regadr in regadrs:
        
        addrrole = regadr.find("addrrole").text
        nameadd = regadr.find("nameadd")
        
        namel = nameadd.find("namel").text if nameadd.find("namel") is not None else None
        addrl = nameadd.find("addrl").text if nameadd.find("addrl") is not None else None
        
        plainco = nameadd.find("plainco").text
        nat = nameadd.find("nat").text
        
        # 1: titulaire
        if int(addrrole) == 1:
            titulaire_name = namel
            titulaire_addrl = addrl
            titulaire_plainco = plainco
            titulaire_nat = nat
        
        # 3: mandataire
        elif int(addrrole) == 3:
            mandataire_namel = namel
            mandataire_addrl = addrl
            mandataire_plainco = plainco
            mandataire_nat = nat
        
    # TODO verify what oppositiongr and oppositionstate mean
    oppositiongr = transac.find("oppositiongr")
    oppositionstate = oppositiongr.find("oppositionstate")

    martype = transac.find("martype").text
    
    # TODO: oppositiongr
    row = [basipikey, origlan, basappn,basregn, basannd, basstate, basregper, expirationdate,
           markve, classes, gstermt, 
           titulaire_name,
           titulaire_addrl,
           titulaire_plainco,
           titulaire_nat,
           mandataire_namel, 
           mandataire_addrl,
           mandataire_plainco,
           mandataire_nat,
           martype]
    
    return row

In [ ]:
def parse_verbal_xml(xml_file):
    import xml.etree.ElementTree as et 
    
    rows = []
    
    # Parse file
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()

    for node in xroot:
        if node.tag == 'transac':
            for transac in node:
                if transac.tag == 'marinfo':
                    # Marques verbales
                    markve = transac.find("markve")
                    if markve is not None:
                        row = retrieve_verbal_data(transac)
                        rows.append(row)
    return rows

In [ ]:
#Use function to parse xml file and save in tables
tables = []
rows = parse_verbal_xml("/content/response.xml")
tables.extend(rows)

In [ ]:
df_verbal = pd.DataFrame(tables, columns=['basipikey', 'origlan', 'basappn','basregn', 'basannd', 
                                   'basstate', 'basregper', 'expirationdate', 'markve', 
                                   'classes', 
                                   'gstermt', 
                                   'titulaire_name',
                                   'titulaire_addrl',
                                   'titulaire_plainco',
                                   'titulaire_nat',
                                   'mandataire_namel', 
                                   'mandataire_addrl',
                                   'mandataire_plainco',
                                   'mandataire_nat',
                                   'martype'])
df_verbal.head()

#columns to drop

###verbal

In [ ]:
df_verbal.drop(['basipikey', 'origlan',  'basannd', 'basstate', 'basregper','gstermt',
       'expirationdate','mandataire_namel','mandataire_addrl', 'mandataire_plainco',
       'mandataire_nat', 'martype'], axis=1, inplace=True)

In [ ]:
#merge columns for df_verbal
df_verbal['titulaire'] = df_verbal.titulaire_name.fillna('') + ' ' + df_verbal.titulaire_addrl.fillna('') + ' ' + df_verbal.titulaire_plainco.fillna('') + ' ' + df_verbal.titulaire_nat.fillna('')

In [ ]:
df_verbal.rename(columns = { 'basappn':'No de la demande', 'basregn':'No de la marque', 'markve':'marque' }, inplace = True)

In [ ]:
df_verbal['Classes de Nice'] = df_verbal['classes'].apply(lambda x: ','.join(map(str, x)))

In [ ]:
df_verbal.drop('classes', axis=1, inplace=True)

In [ ]:
df_verbal.drop(['titulaire_name','titulaire_addrl','titulaire_plainco','titulaire_nat'], axis=1, inplace=True)

In [ ]:
df_verbal['titulaire'] = df_verbal['titulaire'].apply(lambda x: remove_accents(x))

In [ ]:
df_verbal['marque'] = df_verbal['marque'].apply(lambda x: remove_accents(x))

In [ ]:
df_verbal['titulaire'] =  df_verbal['titulaire'].str.replace(r'\n', '', regex=True)


In [ ]:
df_verbal.head()

In [ ]:
df_verbal_html = df_verbal.to_html().replace('<table border="1" class="dataframe">',
                                                              '<table border="1" style="border-collapse: '
                                                              'collapse;text-align: center;font-family:Calibri;font-size:10px;" '
                                                              'class="dataframe">').replace(
                                                              '<tr style="text-align: right;">', '<tr style="text-align: center;table-layout: fixed; width: auto; index:False";>')

In [ ]:
#save html result
with open("df_verbal.html", "w") as f:
    f.write(df_verbal_html)

In [ ]:
pdfkit.from_file('df_verbal.html', 'Marque_verbal.pdf')

##marque

In [ ]:
#display trademark logo in dataframe
# from IPython.core.display import HTML
# def to_img_tag(path):
#     return '<img src="'+ path + '" width="85"  >'

In [ ]:
# display(HTML(df.to_html(escape=False,formatters=dict(Logo=to_img_tag))))

In [ ]:
#function to search for company name and return dataframe


def commmerce_name_search(query):

    username = "bs@legalmarque.com"
    password = "YbxnQ2rt"
    url = 'https://www.zefix.admin.ch/ZefixPublicREST/api/v1/company/search'


    #String Formatting
    post_json = (

    "{"  #first curly bracket
    "\"activeOnly\": true,"  #--> key, value begins and ends with \" except for bool
     f"\"name\": \"{query}\""
     "}" #last curly bracket
     
     )
    #convert formatted string to json
    payload = json.loads(post_json)

    #define headers
    headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
    }

    #post request using above params
    response = requests.post(url, 
                         headers=headers,
                         auth=(username, password), 
                         json=payload)
    
    if response.status_code == 200:
        commerce_search = response.json()
        df = json_normalize(commerce_search)

        #drop unwanted columns
        df.drop(['ehraid','chid', 'legalSeatId',
       'registryOfCommerceId', 'status', 'sogcDate', 'deletionDate',
       'legalForm.id', 'legalForm.uid', 'legalForm.name.de',
       'legalForm.name.fr', 'legalForm.name.it', 'legalForm.name.en',
       'legalForm.shortName.de',
       'legalForm.shortName.it', 'legalForm.shortName.en'], axis=1, inplace=True)
        
        #rename cols with new names
        df[['Raison de commerce', 'IDE', 'Siege', 'Forme']] = df[['name', 'uid','legalSeat','legalForm.shortName.fr']]

        #drop old names
        df.drop(["name", "uid", "legalSeat", "legalForm.shortName.fr"], axis=1, inplace=True)

        return df

    else: 
        print(f"Invalid User Authentification: {response.status_code}")

In [ ]:
df_commerce = commmerce_name_search(query)

In [ ]:
df_commerce.head()

In [ ]:
df_commerce['Siege'] = df_commerce['Siege'].apply(lambda x: remove_accents(x))

In [ ]:
df_commerce['Raison de commerce'] = df_commerce['Raison de commerce'].apply(lambda x: remove_accents(x))

In [ ]:
df_commerce['Forme'] = df_commerce['Forme'].apply(lambda x: remove_accents(x))

In [ ]:
df_commerce_html = df_commerce.to_html().replace('<table border="1" class="dataframe">',
                                                              '<table border="1" style="border-collapse: '
                                                              'collapse;text-align: center;font-family:Calibri;font-size:20px;" '
                                                              'class="dataframe;">').replace(
                                                              '<tr style="text-align: right;">', '<tr style="text-align: center;table-layout: fixed; width: 100%; index:False";>')

In [ ]:
# df_commerce_html = df_commerce.style.set_properties(subset=['Marques', 'IDE','Siege', 'Forme'], **{'width': '200px'}).to_html()
# .replace('<table border="1" class="dataframe">',
#                                                               '<table border="1" style="border-collapse: '
#                                                               'collapse;text-align: center;font-family:Calibri;font-size:20px;" '
#                                                               'class="dataframe;">').replace(
#                                                               '<tr style="text-align: right;">', '<tr style="text-align: center;table-layout: fixed; width: 100%; index:False";>')

In [ ]:
#save html result
with open("df_commerce.html", "w") as f:
    f.write(df_commerce_html)

In [ ]:
pdfkit.from_file('df_commerce.html', 'marque_commerce.pdf')

**DomainName**

In [ ]:
from domainavailability import *

def domain_name_checker(query):

    client = Client('at_kvQSqHPLS7IgXJQiRdDA0o1ePSNBv')

    raw_result_ch = client.raw_data(
    f'{query}' + '.ch',
    mode=Client.DNS_AND_WHOIS_MODE,
    credits_type=Client.WHOIS_CREDITS,
    output_format=Client.XML_FORMAT)

    time.sleep(3)

    raw_result_swiss = client.raw_data(
    f'{query}' + '.swiss',
    mode=Client.DNS_AND_WHOIS_MODE,
    credits_type=Client.WHOIS_CREDITS,
    output_format=Client.XML_FORMAT)

    df_ch = pd.read_xml(raw_result_ch)
    df_swiss = pd.read_xml(raw_result_swiss)

    frames = [df_ch, df_swiss]
    result = pd.concat(frames, ignore_index=True)

    for i in result['domainAvailability']:
        if i == "AVAILABLE":
            result['domainAvailability'].replace(i, 'Disponible', inplace=True)
        else:
            result['domainAvailability'].replace(i, 'Pas Disponible', inplace=True)

    return result




In [ ]:
df_domain = domain_name_checker(query)

In [ ]:
df_domain

In [ ]:
df_domain_html = df_domain.to_html().replace('<table border="1" class="dataframe">',
                                                            '<table border="1" style="border-collapse: '
                                                            'collapse;text-align: center;font-family:Calibri;font-size:20px;" '
                                                            'class="dataframe;">').replace(
                                                            '<tr style="text-align: right;">', '<tr style="text-align: center;table-layout: fixed; width: 100%; index:False";>')
#save html result
with open("domain_Name.html", "w") as f:
    f.write(df_domain_html)

pdfkit.from_file('domain_Name.html', 'domain_Name.pdf')

In [ ]:
from PyPDF2 import PdfMerger

merger = PdfMerger()

for pdf in ["Marque_verbal.pdf", "marque_logo.pdf", "marque_commerce.pdf", "domain_Name.pdf"]:
    merger.append(pdf)

merger.write("merged-pdf.pdf")
merger.close()